In [357]:
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pickle
pd.set_option('display.max_columns', 500)


### Building NMF probability matrix for extrinsic model - will calculate cosine similarities on this!

In [358]:
with open("df_train.pkl", 'rb') as picklefile: 
    df_train = pickle.load(picklefile)

In [359]:
with open("df_sent.pkl", 'rb') as picklefile: 
    df_sent = pickle.load(picklefile)

In [360]:
with open("full_dataset.pkl", 'rb') as picklefile: 
    df = pickle.load(picklefile)

### Training data is smaller subset for speed improvements

In [362]:
X = df_train.sentence
X_Full = df_sent.sentence

### TFIDF on sentences to vectorize, NMF to model
* Max_df is very important!
* Reducing max_features for speed
* Building vectorizer on subset, tranforming all data
* NMF for high number of components (black box, who cares?)
* Again, fit NMF on training, transform all for final results

In [364]:
# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df =.01,max_features=10000, stop_words='english')
tfidf_vectorizer.fit(X)
tfidf = tfidf_vectorizer.transform(X)
tfidf_full = tfidf_vectorizer.transform(X_Full)

In [365]:
# Run NMF
nmf = NMF(n_components=50, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd')
nmf.fit(tfidf)
nmf_p = nmf.transform(tfidf_full)

### Convert probabilities to dataframe & pickle for use in app

In [368]:
#make probability matrix (sentences)
df_sent_pb = pd.DataFrame(nmf_p)
df_sent_pb['id'] = df_sent['id']
df_sent_pb.set_index('id', inplace=True)

In [337]:
#final probability matrix (combine sentences)
df_doc_pb = df_sent_pb.groupby('id').sum()

In [350]:
with open('probability_matrix_10000_50.pkl', 'wb') as picklefile:
    pickle.dump(df_sent_pb, picklefile)

### See rec_routes.py for flask app and use of this model!